In [1]:
import pandas as pd
import numpy as np
import glob as glob
import os

In [2]:
"""### Import Files

Import all files from specified folder and combine into one dataframe called 'df'

has to be one construct but can be multiple plates from diff days etc
"""

path = r'/Users/irishardege/Library/CloudStorage/OneDrive-UniversityofCambridge/Zoology/DATA/TEVC/acr21_lgc4/Tina/data/DRs/c elegans WT' # use your path
all_files = glob.glob(path + "/*_Datatable.xlsx") # use your extension

li = []

for filename in all_files:
    df = pd.read_excel(filename, index_col=None, header=0) # for .dat files you must specify separation for example sep='\t'
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

df.shape
df.head(5)

,Construct,IV Prot.,IVCurveID,Well,ID,Series,ROI,Minimum,Pos Min,Maximum,...,Comp. 1,conc. 1,unit 1,Comp. 2,conc. 2,unit 2,Comp. 3,conc. 3,unit 3,Extremum
0,WT,NaN,NaN,A2,0,0,0,-140,14794,108,...,Dopamine,100,nM,empty,0,mM,empty,0,mM,108
1,WT,NaN,NaN,A2,1,0,0,-145,14882,4,...,Dopamine,1,µM,empty,0,mM,empty,0,mM,4
2,WT,NaN,NaN,A2,2,0,0,-507,20358,5,...,Dopamine,10,µM,empty,0,mM,empty,0,mM,-507
3,WT,NaN,NaN,A2,3,0,0,-1145,20660,3,...,Dopamine,100,µM,empty,0,mM,empty,0,mM,-1145
4,WT,NaN,NaN,A2,4,0,0,-1472,24106,5,...,Dopamine,1,mM,empty,0,mM,empty,0,mM,-1472


In [12]:
"""### Remove excess coloumns and clean data
Here we are keeping only the relevant information, you can add or remove coloumns as you like.
We will also be standardising the units and generally cleaning up the data
"""

#coloumn names must be updated to those in your files, these are based on the standard coloumn titles from Robocyte2+ software export
data =  df[['Well'] + ['Buffer'] + ['Comp. 1'] + ['conc. 1'] + ['unit 1'] + ['Minimum'] + ['Baseline Average'] + ['Start Date'] + ['Construct']] 

# Ensure Start Date is in datetime format
data['Start Date'] = pd.to_datetime(data['Start Date'])

# Extract just the date (as string in yyyy-mm-dd format)
data['date'] = data['Start Date'].dt.date.astype(str)

# Create the unique ID
data['unique_ID'] = data['Construct'] + '_' + data['Well'] + '_' + data['date'] + '_' + data['Comp. 1']

"""Transform units all to uM scale"""

#Transform units all to µM scale
conditions = [
    (data['unit 1'] == 'mM'),
    (data['unit 1'] == 'µM'),
    (data['unit 1'] == 'nM')]

choices = ['1000', '1', '0.001']

data['factor'] = np.select(conditions, choices, default='0')
data['factor']=data.factor.astype(float)
data['Agonist_Concentration_µM'] = data['conc. 1'] * data['factor']

data =  data[['Well'] + ['Buffer'] + ['Comp. 1'] + ['Minimum'] + ['Agonist_Concentration_µM'] + ['date'] + ['unique_ID'] + ['Construct']]
data.reset_index(drop=True)
data.head(5)

/var/folders/dq/ywz90syj7gd0yltqn3vxb0sc0000gn/T/ipykernel_3648/1856731707.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Start Date'] = pd.to_datetime(data['Start Date'])
/var/folders/dq/ywz90syj7gd0yltqn3vxb0sc0000gn/T/ipykernel_3648/1856731707.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data['Start Date'].dt.date.astype(str)
/var/folders/dq/ywz90syj7gd0yltqn3vxb0sc0000gn/T/ipykernel_3648/1856731707.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy o

,Well,Buffer,Comp. 1,Minimum,Agonist_Concentration_µM,date,unique_ID,Construct
0,A2,empty,Dopamine,-140,0.1,2025-09-05,WT_A2_2025-09-05_Dopamine,WT
1,A2,empty,Dopamine,-145,1.0,2025-09-05,WT_A2_2025-09-05_Dopamine,WT
2,A2,empty,Dopamine,-507,10.0,2025-09-05,WT_A2_2025-09-05_Dopamine,WT
3,A2,empty,Dopamine,-1145,100.0,2025-09-05,WT_A2_2025-09-05_Dopamine,WT
4,A2,empty,Dopamine,-1472,1000.0,2025-09-05,WT_A2_2025-09-05_Dopamine,WT


In [ ]:
#OPTIONAL keep one compound only

#data = data[data['Comp. 1'] == 'Dopamine']
#data.head(5)

,Well,Buffer,Comp. 1,Minimum,Agonist_Concentration_µM,date,unique_ID,Construct
0,A2,empty,Dopamine,-140,0.1,2025-09-05,WT_A2_2025-09-05_Dopamine,WT
1,A2,empty,Dopamine,-145,1.0,2025-09-05,WT_A2_2025-09-05_Dopamine,WT
2,A2,empty,Dopamine,-507,10.0,2025-09-05,WT_A2_2025-09-05_Dopamine,WT
3,A2,empty,Dopamine,-1145,100.0,2025-09-05,WT_A2_2025-09-05_Dopamine,WT
4,A2,empty,Dopamine,-1472,1000.0,2025-09-05,WT_A2_2025-09-05_Dopamine,WT
5,A2,empty,Dopamine,-1319,10000.0,2025-09-05,WT_A2_2025-09-05_Dopamine,WT
11,A3,empty,Dopamine,-160,0.1,2025-09-05,WT_A3_2025-09-05_Dopamine,WT
12,A3,empty,Dopamine,-170,1.0,2025-09-05,WT_A3_2025-09-05_Dopamine,WT
13,A3,empty,Dopamine,-572,10.0,2025-09-05,WT_A3_2025-09-05_Dopamine,WT
14,A3,empty,Dopamine,-1757,100.0,2025-09-05,WT_A3_2025-09-05_Dopamine,WT


In [13]:
#data['log_Agonist_µM'] = np.log(data['Agonist_Concentration_µM']) #if you wish to calculate log concentration
data['Agonist'] = data['Comp. 1']
data['Response'] = data['Minimum']
data = data.drop(columns=['Comp. 1'])
data.head()

"""## Transform data to I/Imax for each oocyte"""

norm = data.Minimum / data.groupby(['unique_ID']).Minimum.transform(np.min)
data['Response'] = norm
normdata = data.drop(columns=['Minimum', 'Buffer']) #drop unwanted coloumns
normgroup = normdata.groupby(['Agonist_Concentration_µM']).describe()

grouped_data = normdata.groupby(['Construct', 'Agonist', 'Agonist_Concentration_µM']) #view grouped data
export_group = grouped_data['Response'].describe()

In [14]:
"""## Export as .csv
comment out if you do not want to export .csv
change path and file name before running

exports grouped mean, std, N into a new .csv file

Includes *injection ID*, Agonist and log agonist concentrations in uM
"""

export_data = normdata
export_mean = export_group

output_folder = os.path.join(path, 'outputs')
os.makedirs(output_folder, exist_ok=True)

output_file = os.path.join(output_folder, 'DR_norm_mean.csv')
export_mean.to_csv(output_file, index=True)

output_file = os.path.join(output_folder, 'DR_norm.csv')
export_data.to_csv(output_file, index=True)